<a href="https://colab.research.google.com/github/jjmcnelis/VegMapper/blob/devel/gee/vegMapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vegMapper

https://github.com/NaiaraSPinto/VegMapper

## TO-DO (2021-05-26)

* Buffer Feature Collection before getting the zonal statistics.

## Setup

After common packages are imported (e.g. *ee*, *numpy*, *pandas*, *matplotlib* -- all are available by default in Google Colab), the next cell will attempt to perform some important housekeeping steps.

### Requirements

This notebook was developed in Python 3.8; it has several dependencies:

* *ee* (Earth Engine Python API)
* *numpy*
* *pandas*
* *matplotlib*
* *geemap*

_geemap_ is the only package that is not available by default in Google Colab. If _geemap_ is not available, the next cell will try to install it with pip and import again.

In [4]:
from IPython.display import HTML
from json import dumps, load
from io import StringIO
#import matplotlib.pyplot as plt
#import numpy as np
import pandas as pd
import os.path
import ee
try:
    #!python -m pip install -q git+git://github.com/geopandas/geopandas.git  # Read binary? Only csv+json?
    #import geopandas as gpd
    !python -m pip install -q geemap
    import geemap
except ImportError as e:
    raise e

### User configuration

Configure the workflow parameters in the cell below. These settings determine how the workflow selects and operates on the data/imagery.

In [91]:
#@markdown Time coverage of imagery selected for the stack:
startDate = '2017-04-01'  #@param {type: "date"}
endDate = '2017-09-30'  #@param {type: "date"}
#@markdown Scale/resolution of the output raster stack, in meters:
scale = 30  #@param {type: "integer"}
#@markdown Buffer distance around inputs xy locations, in meters:
buffer = 30  #@param {type: "integer"}
#@markdown The DEBUG ON option causes the ipynb to run much slower, runs stats and renders maps.
debug = "DEBUG OFF (fast)" #@param ["DEBUG ON (slow)", "DEBUG OFF (fast)"] {allow-input: true}

def debug_status():
    return "ON" in debug and "OFF" not in debug

#debug_status()

### Authenticate for GEE and Google Drive

>**Quickstart:**
>Run the next cell and follow the instructions to authenticate. Click the links displayed below the cell and log in (each one should open in a new browser tab); copy your temporary token, then paste it into the prompt and press enter.

(I can probably merge the auth together with a little digging into the GEE/Colab APIs, but for now) expect to see two prompts for your Google login info:
1. for GEE access (*REQUIRED*)
2. for Google Drive access (*OPTIONAL*, only available in Colab)

You can upload/download to the Colab environment in one of (at least) two ways if Google Drive is not accessible (i.e. no space remaining): 
1. using the File Manager (on the left in the Colab interface), or 
2. using interactive prompts as you progress through the notebook.

However, if you're running this notebook *outside* of the Colab environment (i.e. in the common Jupyter notebook client) then you will need to call *pandas* manually to read/write the input & output (with `pd.read_csv` and `<df>.to_csv`, respectively).

In [6]:
####################################################################
# My GEE credentials are tied to a personal Google account. The 1st
# prompt provides a link opening a new tab to the familiar Google 
# login page. After logging in, copy the token into the prompt and
# hit enter. This step is required in order to proceed.
####################################################################
ee.Authenticate()
ee.Initialize()

####################################################################
# This next part checks if the notebook is running in Colab first. 
# If so, you will be prompted to log in *again* for Google Drive 
# access. Run the next cell to skip mounting Drive into Colab.
####################################################################
DRIVE = "/content/drive/MyDrive"
#DRIVE = "/content/drive/Shareddrives"
if 'google.colab' in str(get_ipython()):
    from google.colab import drive, files
    try:
        drive.mount("/content/drive")
    except Exception as e:
        print("Next cell executed. Will skip mounting Drive.")

####################################################################
# Housekeeping -- please ignore the remainder of this cell
####################################################################

def _validate_path_and_read_input_csv_data():
    # If input csv path is 'None', prompt the user for upload.
    if not relative_path_to_input_csv_in_drive:
        csv = None
    elif not os.path.isdir(DRIVE):
        # Error out if path is not provided and Drive isnt mounted.
        raise Exception("ERROR: Cannot determine if Drive is mounted.")
    else:
        # Otherwise make sure the input csv path is valid.
        if os.path.isfile(relative_path_to_input_csv_in_drive):
            csv = relative_path_to_input_csv_in_drive
        else:
            # Assume its an absolute path if the relative path is invalid.
            csv = os.path.join(DRIVE, relative_path_to_input_csv_in_drive)
    if not csv:
        # Prompt the user to upload their csv if 'csv' is 'None'.
        uploads = files.upload()
        if len(uploads)==0:
            raise Exception("ERROR: Received no files. Try again.")
        elif len(uploads)>1:
            raise Exception("ERROR: Received multiple files. Try again.")
        else:
            csv = list(uploads)[0]
    # Finally, attempt to load the input csv to a pandas data frame:
    try:
        df = pd.read_csv(csv)
        display(df.info())
    except FileNotFoundError as e:
        raise Exception(".")
    except Exception as e:
        raise e
    return df

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=I2HGtUham-vtDwAV3BXKn0aYY0RY_vMxmgevqS0ZcJE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g44x3hdASdm7OUrH3-ynskIt4-UtgeNP3oprZa5JcP8xkZzPkWDFTE

Successfully saved authorization token.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load input data

As mentioned before, this procedure builds a stack of images and calculates the zonal statistics within regions defined by an input feature dataset. 

*CSV is the only supported file format for the input xy point locations.* It should have these columns at minimum:
* *latitude* (float)
* *longitude* (float)

The following cell will read your input table of XY positions (which should be provided in the *latitude* and *longitude* columns) and any additional data columns then print some high level details (assuming your paths are configured properly).

#### Option a: read files from Google Drive

You should see a folder "drive/" in the default workspace when you open the file browser panel (to the immediate left inside the Colab environment).

><u>A note about paths configured in the next cell:</u>    
>Input and output paths should be set relative to the root of the _drive_ directory shown _OR_, alternatively, you can provide absolute paths to your input/output file(s).

Try to remember to unmount Drive once you're finished in Colaboratory. You can do that by calling this other function from the drive module: `google.colab.drive.flush_and_unmount()`

#### Option b: upload/read files to Colaboratory

*The workflow requires input features to determine the areas in which to calculate zonal statistics.*

Make sure a suitable file exists in the colab workspace or in Google Drive. You can provide one in either of two ways:

1. Navigate to an input CSV in Google Drive and copy its path into the cell below (assuming Drive is mounted), or
2. Run the next cell as-is and upload a file to the workspace when prompted.

If the second option, run the next cell and click *Choose Files* to upload a file. You may also click *Cancel Upload* to abort the cell and move on.

In [7]:
# Set this to 'None' to be prompted to upload your input csv:
relative_path_to_input_csv_in_drive = f"tests/vegMapper/smartin.csv"

# Set this to 'None' to be prompted to download your outputs csv: 
relative_path_to_output_csv_in_drive = f"tests/vegMapper/out/outputs.csv"

### Error out here if any inputs are invalid >>>
pts = _validate_path_and_read_input_csv_data()

print("Success! Please proceed with the notebook.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   longitude   100 non-null    float64
 1   latitude    100 non-null    float64
 2   obs_year    50 non-null     float64
 3   class       50 non-null     object 
 4   class_2017  100 non-null    object 
dtypes: float64(3), object(2)
memory usage: 4.0+ KB


None

Success! Please proceed with the notebook.


#### Get xy point geometries as a Feature Collection

Make geometries for each XY position in the input table so we can efficiently generate zonal statistics over our final image stack at the end of the procedure. See the API documentation for information about [*ee.FeatureCollection*](>https://developers.google.com/earth-engine/guides/feature_collections)s.

In [8]:
def get_geom(x):
    return ee.Geometry.Point(x['longitude'], x['latitude'])

pfc = ee.FeatureCollection(pts.apply(get_geom, axis=1).tolist())

type(pfc)

ee.featurecollection.FeatureCollection


### Load roi json as a Geometry Polygon

Otherwise, use the minimum extent of the points in the input CSV. Add an arbitrary buffer around the minimum extent and then get a *ee.Geometry.Polygon* to represent the ROI.

In [10]:
%%file drive/MyDrive/tests/vegMapper/smartin.geojson
{
    "type": "FeatureCollection",
    "name": "ucayali_shape",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": [
    { "type": "Feature", "properties": { "FIPS_ADMIN": "PE25", "GMI_ADMIN": "PER-UCY", "ADMIN_NAME": "Ucayali", "FIPS_CNTRY": "PE", "GMI_CNTRY": "PER", "CNTRY_NAME": "Peru", "REGION": "South America", "CONTINENT": "South America", "POP_ADMIN": 256700, "SQKM_ADMIN": 104382.602, "SQMI_ADMIN": 40302.121, "TYPE_ENG": "Department", "TYPE_LOC": "Departamento", "COLOR_MAP": "3" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -70.628616333007812, -9.948488235473633 ], [ -70.644729614257812, -9.956390380859375 ], [ -70.687225341796875, -9.983612060546875 ], [ -70.799453735351562, -10.066389083862305 ], [ -70.805557250976562, -10.074167251586914 ], [ -70.815292358398438, -10.091945648193359 ], [ -70.829177856445312, -10.134445190429688 ], [ -70.835556030273438, -10.156946182250977 ], [ -70.837783813476562, -10.168611526489258 ], [ -70.8416748046875, -10.227500915527344 ], [ -70.8416748046875, -10.248056411743164 ], [ -70.844451904296875, -10.294724464416504 ], [ -70.845840454101562, -10.30694580078125 ], [ -70.850845336914062, -10.343889236450195 ], [ -70.852783203125, -10.356111526489258 ], [ -70.861953735351562, -10.375278472900391 ], [ -70.867507934570312, -10.383611679077148 ], [ -70.9808349609375, -10.521112442016602 ], [ -70.98779296875, -10.528057098388672 ], [ -71.017791748046875, -10.552501678466797 ], [ -71.105560302734375, -10.618890762329102 ], [ -71.210006713867188, -10.695556640625 ], [ -71.245285034179688, -10.71611213684082 ], [ -71.275283813476562, -10.727779388427734 ], [ -71.298065185546875, -10.732500076293945 ], [ -71.320556640625, -10.738611221313477 ], [ -71.351959228515625, -10.748889923095703 ], [ -71.37139892578125, -10.75694465637207 ], [ -71.438613891601562, -10.787223815917969 ], [ -71.486114501953125, -10.809722900390625 ], [ -71.605010986328125, -10.873889923095703 ], [ -71.73028564453125, -10.943056106567383 ], [ -71.7569580078125, -10.958612442016602 ], [ -71.765838623046875, -10.963335037231445 ], [ -71.7852783203125, -10.972501754760742 ], [ -71.804733276367188, -10.980555534362793 ], [ -71.828338623046875, -10.984724044799805 ], [ -71.84112548828125, -10.986112594604492 ], [ -71.9041748046875, -10.990835189819336 ], [ -71.932235717773438, -10.990835189819336 ], [ -71.9586181640625, -10.990835189819336 ], [ -71.9969482421875, -10.984724044799805 ], [ -72.029449462890625, -10.956111907958984 ], [ -72.041946411132812, -10.947778701782227 ], [ -72.067230224609375, -10.934167861938477 ], [ -72.1158447265625, -10.921112060546875 ], [ -72.13250732421875, -10.926668167114258 ], [ -72.14361572265625, -10.9375 ], [ -72.147781372070312, -10.951112747192383 ], [ -72.147232055664062, -10.964723587036133 ], [ -72.14306640625, -10.97972297668457 ], [ -72.1219482421875, -11.005834579467773 ], [ -72.11083984375, -11.015277862548828 ], [ -72.080291748046875, -11.051668167114258 ], [ -72.080291748046875, -11.069168090820312 ], [ -72.083343505859375, -11.085835456848145 ], [ -72.111679077148438, -11.12611198425293 ], [ -72.139450073242188, -11.164167404174805 ], [ -72.156402587890625, -11.190279006958008 ], [ -72.171401977539062, -11.231111526489258 ], [ -72.180557250976562, -11.273335456848145 ], [ -72.18194580078125, -11.293889999389648 ], [ -72.180419921875, -11.313259124755859 ], [ -72.206390380859375, -11.302778244018555 ], [ -72.218063354492188, -11.300834655761719 ], [ -72.296676635742188, -11.289167404174805 ], [ -72.406112670898438, -11.324722290039062 ], [ -72.413619995117188, -11.330833435058594 ], [ -72.459457397460938, -11.369722366333008 ], [ -72.471389770507812, -11.385278701782227 ], [ -72.530563354492188, -11.420000076293945 ], [ -72.540283203125, -11.42500114440918 ], [ -72.568893432617188, -11.434446334838867 ], [ -72.58056640625, -11.433055877685547 ], [ -72.62445068359375, -11.420833587646484 ], [ -72.690567016601562, -11.393611907958984 ], [ -72.698898315429688, -11.387500762939453 ], [ -72.833892822265625, -11.375 ], [ -72.908889770507812, -11.336112976074219 ], [ -72.916671752929688, -11.330001831054688 ], [ -72.939727783203125, -11.311111450195312 ], [ -72.9666748046875, -11.249444961547852 ], [ -72.970291137695312, -11.238611221313477 ], [ -73.03973388671875, -11.252223968505859 ], [ -73.046112060546875, -11.260557174682617 ], [ -73.053070068359375, -11.267223358154297 ], [ -73.060836791992188, -11.273335456848145 ], [ -73.071395874023438, -11.277500152587891 ], [ -73.091400146484375, -11.284444808959961 ], [ -73.1239013671875, -11.293889999389648 ], [ -73.15972900390625, -11.30000114440918 ], [ -73.231948852539062, -11.311111450195312 ], [ -73.256393432617188, -11.314444541931152 ], [ -73.404449462890625, -11.334166526794434 ], [ -73.41778564453125, -11.333612442016602 ], [ -73.4283447265625, -11.329444885253906 ], [ -73.435195922851562, -11.324211120605469 ], [ -73.455291748046875, -11.31361198425293 ], [ -73.46917724609375, -11.299446105957031 ], [ -73.48028564453125, -11.283056259155273 ], [ -73.490005493164062, -11.26472282409668 ], [ -73.508895874023438, -11.214168548583984 ], [ -73.5150146484375, -11.192779541015625 ], [ -73.516677856445312, -11.180557250976562 ], [ -73.516677856445312, -11.17388916015625 ], [ -73.514450073242188, -11.153890609741211 ], [ -73.504730224609375, -11.107500076293945 ], [ -73.4908447265625, -11.038612365722656 ], [ -73.490005493164062, -11.025001525878906 ], [ -73.49139404296875, -11.012500762939453 ], [ -73.494171142578125, -11.002500534057617 ], [ -73.511123657226562, -10.964168548583984 ], [ -73.538894653320312, -10.908889770507812 ], [ -73.560287475585938, -10.874723434448242 ], [ -73.571670532226562, -10.858333587646484 ], [ -73.602783203125, -10.819168090820312 ], [ -73.633346557617188, -10.793889999389648 ], [ -73.642501831054688, -10.789167404174805 ], [ -73.760833740234375, -10.745555877685547 ], [ -73.7711181640625, -10.760557174682617 ], [ -73.788619995117188, -10.795278549194336 ], [ -73.8094482421875, -10.836389541625977 ], [ -73.84722900390625, -10.888889312744141 ], [ -73.874176025390625, -10.925834655761719 ], [ -73.885284423828125, -10.940834045410156 ], [ -73.899169921875, -10.954444885253906 ], [ -73.911956787109375, -10.964168548583984 ], [ -73.9283447265625, -10.972501754760742 ], [ -73.9425048828125, -10.977779388427734 ], [ -73.9727783203125, -10.977779388427734 ], [ -73.989730834960938, -10.973056793212891 ], [ -74.018890380859375, -10.967500686645508 ], [ -74.109451293945312, -10.981111526489258 ], [ -74.153900146484375, -10.992778778076172 ], [ -74.16778564453125, -10.998334884643555 ], [ -74.237228393554688, -11.021389007568359 ], [ -74.377227783203125, -11.055000305175781 ], [ -74.395278930664062, -11.057779312133789 ], [ -74.4091796875, -11.056390762329102 ], [ -74.423065185546875, -11.050834655761719 ], [ -74.43695068359375, -11.042778015136719 ], [ -74.449722290039062, -11.033889770507812 ], [ -74.4727783203125, -10.996946334838867 ], [ -74.479446411132812, -10.983333587646484 ], [ -74.48779296875, -10.956111907958984 ], [ -74.49639892578125, -10.912223815917969 ], [ -74.508712768554688, -10.848185539245605 ], [ -74.468902587890625, -10.830278396606445 ], [ -74.400848388671875, -10.772223472595215 ], [ -74.388336181640625, -10.755834579467773 ], [ -74.357223510742188, -10.720001220703125 ], [ -74.287506103515625, -10.651668548583984 ], [ -74.230560302734375, -10.622501373291016 ], [ -74.20001220703125, -10.61277961730957 ], [ -74.170562744140625, -10.607224464416504 ], [ -74.164459228515625, -10.595001220703125 ], [ -74.16278076171875, -10.580001831054688 ], [ -74.181121826171875, -10.555278778076172 ], [ -74.192230224609375, -10.545833587646484 ], [ -74.20611572265625, -10.538888931274414 ], [ -74.249176025390625, -10.522500991821289 ], [ -74.264724731445312, -10.519723892211914 ], [ -74.282791137695312, -10.513612747192383 ], [ -74.2952880859375, -10.505556106567383 ], [ -74.301956176757812, -10.490278244018555 ], [ -74.354171752929688, -10.361667633056641 ], [ -74.371124267578125, -10.315279006958008 ], [ -74.37445068359375, -10.30000114440918 ], [ -74.37445068359375, -10.268611907958984 ], [ -74.370285034179688, -10.221944808959961 ], [ -74.360000610351562, -10.1602783203125 ], [ -74.3558349609375, -10.143890380859375 ], [ -74.360000610351562, -10.11250114440918 ], [ -74.372222900390625, -10.085000991821289 ], [ -74.390289306640625, -10.055000305175781 ], [ -74.411392211914062, -10.024723052978516 ], [ -74.432785034179688, -9.981668472290039 ], [ -74.438613891601562, -9.966390609741211 ], [ -74.441390991210938, -9.952777862548828 ], [ -74.441390991210938, -9.936389923095703 ], [ -74.43695068359375, -9.90333366394043 ], [ -74.432785034179688, -9.88972282409668 ], [ -74.43084716796875, -9.87611198425293 ], [ -74.446121215820312, -9.828056335449219 ], [ -74.465560913085938, -9.774444580078125 ], [ -74.481674194335938, -9.731945037841797 ], [ -74.498336791992188, -9.701946258544922 ], [ -74.512222290039062, -9.677223205566406 ], [ -74.535842895507812, -9.651111602783203 ], [ -74.538253784179688, -9.64991569519043 ], [ -74.58111572265625, -9.580556869506836 ], [ -74.658340454101562, -9.466667175292969 ], [ -74.665283203125, -9.459722518920898 ], [ -74.673065185546875, -9.453611373901367 ], [ -74.68695068359375, -9.439167022705078 ], [ -74.699447631835938, -9.42388916015625 ], [ -74.70916748046875, -9.406112670898438 ], [ -74.712783813476562, -9.395833969116211 ], [ -74.718902587890625, -9.373889923095703 ], [ -74.720840454101562, -9.362222671508789 ], [ -74.720291137695312, -9.348611831665039 ], [ -74.706390380859375, -9.29222297668457 ], [ -74.668624877929688, -9.135557174682617 ], [ -74.618621826171875, -8.899168014526367 ], [ -74.559707641601562, -8.785251617431641 ], [ -74.513565063476562, -8.759613037109375 ], [ -74.510284423828125, -8.749168395996094 ], [ -74.508895874023438, -8.728612899780273 ], [ -74.51556396484375, -8.695001602172852 ], [ -74.522781372070312, -8.673612594604492 ], [ -74.526947021484375, -8.664722442626953 ], [ -74.532501220703125, -8.656389236450195 ], [ -74.539459228515625, -8.649444580078125 ], [ -74.547225952148438, -8.642501831054688 ], [ -74.563064575195312, -8.631668090820312 ], [ -74.573623657226562, -8.627500534057617 ], [ -74.611953735351562, -8.622777938842773 ], [ -74.657791137695312, -8.619167327880859 ], [ -74.677230834960938, -8.618612289428711 ], [ -74.701400756835938, -8.621389389038086 ], [ -74.723892211914062, -8.626945495605469 ], [ -74.79278564453125, -8.654445648193359 ], [ -74.802505493164062, -8.658334732055664 ], [ -74.812225341796875, -8.663333892822266 ], [ -74.984176635742188, -8.782222747802734 ], [ -75.046676635742188, -8.843612670898438 ], [ -75.0614013671875, -8.858612060546875 ], [ -75.0675048828125, -8.866111755371094 ], [ -75.160568237304688, -8.996667861938477 ], [ -75.212783813476562, -9.096944808959961 ], [ -75.233901977539062, -9.14666748046875 ], [ -75.2952880859375, -9.231111526489258 ], [ -75.3013916015625, -9.239168167114258 ], [ -75.308334350585938, -9.246112823486328 ], [ -75.326400756835938, -9.255001068115234 ], [ -75.355010986328125, -9.267501831054688 ], [ -75.508895874023438, -9.347223281860352 ], [ -75.555557250976562, -9.384166717529297 ], [ -75.582504272460938, -9.398612976074219 ], [ -75.59222412109375, -9.402778625488281 ], [ -75.614456176757812, -9.408334732055664 ], [ -75.625564575195312, -9.4102783203125 ], [ -75.63250732421875, -9.4102783203125 ], [ -75.644180297851562, -9.408334732055664 ], [ -75.661956787109375, -9.397222518920898 ], [ -75.668624877929688, -9.390277862548828 ], [ -75.783065795898438, -9.238611221313477 ], [ -75.787506103515625, -9.22972297668457 ], [ -75.797500610351562, -9.198890686035156 ], [ -75.801116943359375, -9.174722671508789 ], [ -75.80029296875, -9.160833358764648 ], [ -75.794723510742188, -9.138334274291992 ], [ -75.793899536132812, -9.125278472900391 ], [ -75.794723510742188, -9.11277961730957 ], [ -75.802230834960938, -9.079168319702148 ], [ -75.81500244140625, -9.05027961730957 ], [ -75.819732666015625, -9.041389465332031 ], [ -75.825836181640625, -9.033889770507812 ], [ -75.841949462890625, -9.022222518920898 ], [ -75.889450073242188, -8.90472412109375 ], [ -75.930557250976562, -8.751945495605469 ], [ -75.965011596679688, -8.699722290039062 ], [ -75.97222900390625, -8.686668395996094 ], [ -75.978347778320312, -8.659168243408203 ], [ -75.974166870117188, -8.645278930664062 ], [ -75.960845947265625, -8.641389846801758 ], [ -75.872512817382812, -8.60472297668457 ], [ -75.858612060546875, -8.595277786254883 ], [ -75.8477783203125, -8.585556030273438 ], [ -75.829452514648438, -8.562223434448242 ], [ -75.768341064453125, -8.497501373291016 ], [ -75.645004272460938, -8.385833740234375 ], [ -75.627792358398438, -8.380556106567383 ], [ -75.5997314453125, -8.380556106567383 ], [ -75.585845947265625, -8.385833740234375 ], [ -75.560836791992188, -8.402500152587891 ], [ -75.53778076171875, -8.423055648803711 ], [ -75.514175415039062, -8.445278167724609 ], [ -75.490570068359375, -8.465000152587891 ], [ -75.45306396484375, -8.487222671508789 ], [ -75.437789916992188, -8.488611221313477 ], [ -75.422500610351562, -8.487222671508789 ], [ -75.410003662109375, -8.474723815917969 ], [ -75.408340454101562, -8.458333969116211 ], [ -75.414169311523438, -8.441667556762695 ], [ -75.422500610351562, -8.429166793823242 ], [ -75.441116333007812, -8.405279159545898 ], [ -75.465560913085938, -8.379722595214844 ], [ -75.476119995117188, -8.370000839233398 ], [ -75.494171142578125, -8.342500686645508 ], [ -75.501113891601562, -8.328056335449219 ], [ -75.513626098632812, -8.286945343017578 ], [ -75.510009765625, -8.256389617919922 ], [ -75.507232666015625, -8.242778778076172 ], [ -75.4989013671875, -8.230278015136719 ], [ -75.331954956054688, -8.105001449584961 ], [ -75.267227172851562, -8.116111755371094 ], [ -75.20306396484375, -8.124168395996094 ], [ -75.187225341796875, -8.124168395996094 ], [ -75.1461181640625, -8.11250114440918 ], [ -75.116958618164062, -8.104167938232422 ], [ -75.090560913085938, -8.086389541625977 ], [ -75.079452514648438, -8.075279235839844 ], [ -75.055557250976562, -8.058055877685547 ], [ -75.041671752929688, -8.049167633056641 ], [ -75.0, -8.026390075683594 ], [ -74.930557250976562, -8.005834579467773 ], [ -74.868621826171875, -7.994722366333008 ], [ -74.839447021484375, -7.991389274597168 ], [ -74.704452514648438, -7.970000267028809 ], [ -74.690567016601562, -7.967223167419434 ], [ -74.65972900390625, -7.95472240447998 ], [ -74.605560302734375, -7.924445152282715 ], [ -74.58056640625, -7.909444808959961 ], [ -74.567779541015625, -7.895556449890137 ], [ -74.5594482421875, -7.883055686950684 ], [ -74.5533447265625, -7.869444847106934 ], [ -74.5513916015625, -7.850556373596191 ], [ -74.54779052734375, -7.762500762939453 ], [ -74.561676025390625, -7.574167251586914 ], [ -74.564453125, -7.555000305175781 ], [ -74.57000732421875, -7.523056030273438 ], [ -74.585845947265625, -7.438889503479004 ], [ -74.59417724609375, -7.411389350891113 ], [ -74.603897094726562, -7.397500038146973 ], [ -74.611114501953125, -7.385278701782227 ], [ -74.614456176757812, -7.366389274597168 ], [ -74.608901977539062, -7.346389770507812 ], [ -74.5997314453125, -7.332777976989746 ], [ -74.588897705078125, -7.323056221008301 ], [ -74.576126098632812, -7.320278167724609 ], [ -74.55694580078125, -7.318889617919922 ], [ -74.512222290039062, -7.327777862548828 ], [ -74.48583984375, -7.34444522857666 ], [ -74.473342895507812, -7.354166984558105 ], [ -74.450836181640625, -7.374167442321777 ], [ -74.435836791992188, -7.383889198303223 ], [ -74.4122314453125, -7.395556449890137 ], [ -74.394180297851562, -7.398333549499512 ], [ -74.359176635742188, -7.401667594909668 ], [ -74.329727172851562, -7.400278091430664 ], [ -74.313339233398438, -7.398333549499512 ], [ -74.2791748046875, -7.398889541625977 ], [ -74.260284423828125, -7.401667594909668 ], [ -74.218063354492188, -7.41694450378418 ], [ -74.089736938476562, -7.467223167419434 ], [ -74.060562133789062, -7.478889465332031 ], [ -74.0211181640625, -7.500278472900391 ], [ -74.009170532226562, -7.507223129272461 ], [ -73.996673583984375, -7.517778396606445 ], [ -73.987152099609375, -7.528833389282227 ], [ -73.9969482421875, -7.526945114135742 ], [ -74.00250244140625, -7.52833366394043 ], [ -74.0069580078125, -7.531111717224121 ], [ -74.009445190429688, -7.535000801086426 ], [ -74.01055908203125, -7.541389465332031 ], [ -74.009735107421875, -7.547500610351562 ], [ -74.007781982421875, -7.55250072479248 ], [ -74.00445556640625, -7.55583381652832 ], [ -73.99639892578125, -7.561667442321777 ], [ -73.974166870117188, -7.574444770812988 ], [ -73.932510375976562, -7.609167098999023 ], [ -73.868621826171875, -7.671389579772949 ], [ -73.862503051757812, -7.679166793823242 ], [ -73.854446411132812, -7.69194507598877 ], [ -73.849456787109375, -7.707222938537598 ], [ -73.843902587890625, -7.715556144714355 ], [ -73.840560913085938, -7.71916675567627 ], [ -73.828903198242188, -7.728333473205566 ], [ -73.814178466796875, -7.735000610351562 ], [ -73.751953125, -7.75694465637207 ], [ -73.724166870117188, -7.764445304870605 ], [ -73.714447021484375, -7.768611907958984 ], [ -73.710556030273438, -7.771389007568359 ], [ -73.70361328125, -7.778611183166504 ], [ -73.69805908203125, -7.786945343017578 ], [ -73.694732666015625, -7.797500610351562 ], [ -73.692779541015625, -7.80916690826416 ], [ -73.694732666015625, -7.835556030273438 ], [ -73.695846557617188, -7.841667175292969 ], [ -73.697235107421875, -7.847222328186035 ], [ -73.705001831054688, -7.860278129577637 ], [ -73.713897705078125, -7.872500419616699 ], [ -73.720840454101562, -7.879167556762695 ], [ -73.725006103515625, -7.88194465637207 ], [ -73.73028564453125, -7.883889198303223 ], [ -73.73638916015625, -7.884445190429688 ], [ -73.741958618164062, -7.883055686950684 ], [ -73.7711181640625, -7.870000839233398 ], [ -73.777236938476562, -7.869167327880859 ], [ -73.7822265625, -7.871111869812012 ], [ -73.7852783203125, -7.875278472900391 ], [ -73.786956787109375, -7.887500762939453 ], [ -73.771957397460938, -7.948056221008301 ], [ -73.764724731445312, -7.961389541625977 ], [ -73.761123657226562, -7.965000152587891 ], [ -73.7569580078125, -7.967778205871582 ], [ -73.742507934570312, -7.974166870117188 ], [ -73.714736938476562, -7.981667518615723 ], [ -73.705001831054688, -7.986111640930176 ], [ -73.655288696289062, -8.013334274291992 ], [ -73.651123046875, -8.016111373901367 ], [ -73.645004272460938, -8.023889541625977 ], [ -73.598617553710938, -8.114168167114258 ], [ -73.592788696289062, -8.128612518310547 ], [ -73.556671142578125, -8.249168395996094 ], [ -73.541397094726562, -8.302778244018555 ], [ -73.5372314453125, -8.339445114135742 ], [ -73.535003662109375, -8.350557327270508 ], [ -73.533065795898438, -8.355556488037109 ], [ -73.527511596679688, -8.363889694213867 ], [ -73.478622436523438, -8.398612976074219 ], [ -73.40472412109375, -8.450555801391602 ], [ -73.348068237304688, -8.5977783203125 ], [ -73.343338012695312, -8.606945037841797 ], [ -73.28167724609375, -8.670278549194336 ], [ -73.273345947265625, -8.675834655761719 ], [ -73.256668090820312, -8.680557250976562 ], [ -73.244171142578125, -8.679166793823242 ], [ -73.23779296875, -8.680000305175781 ], [ -73.215560913085938, -8.686111450195312 ], [ -73.190567016601562, -8.696111679077148 ], [ -73.177230834960938, -8.704168319702148 ], [ -73.169723510742188, -8.71027946472168 ], [ -73.163619995117188, -8.718055725097656 ], [ -73.152511596679688, -8.735000610351562 ], [ -73.142501831054688, -8.759445190429688 ], [ -73.141403198242188, -8.76500129699707 ], [ -73.132781982421875, -8.790555953979492 ], [ -73.1219482421875, -8.807500839233398 ], [ -73.065567016601562, -8.895000457763672 ], [ -73.062789916992188, -8.899168014526367 ], [ -73.055557250976562, -8.906389236450195 ], [ -73.0513916015625, -8.909168243408203 ], [ -73.011947631835938, -8.925834655761719 ], [ -73.003067016601562, -8.931390762329102 ], [ -72.996124267578125, -8.938333511352539 ], [ -72.96417236328125, -8.983333587646484 ], [ -72.960556030273438, -8.993612289428711 ], [ -72.951126098632812, -9.045833587646484 ], [ -72.94805908203125, -9.063056945800781 ], [ -72.94805908203125, -9.070001602172852 ], [ -72.948333740234375, -9.083889007568359 ], [ -72.95001220703125, -9.096389770507812 ], [ -72.955291748046875, -9.119167327880859 ], [ -72.966400146484375, -9.143056869506836 ], [ -73.02362060546875, -9.221111297607422 ], [ -73.11944580078125, -9.313333511352539 ], [ -73.158065795898438, -9.344446182250977 ], [ -73.178070068359375, -9.359445571899414 ], [ -73.185012817382812, -9.366111755371094 ], [ -73.190567016601562, -9.375 ], [ -73.199447631835938, -9.394166946411133 ], [ -73.205291748046875, -9.407222747802734 ], [ -73.198623657226562, -9.407501220703125 ], [ -72.9808349609375, -9.410833358764648 ], [ -72.861953735351562, -9.412223815917969 ], [ -72.840835571289062, -9.411666870117188 ], [ -72.730010986328125, -9.413057327270508 ], [ -72.606948852539062, -9.446111679077148 ], [ -72.402236938476562, -9.483612060546875 ], [ -72.376953125, -9.490835189819336 ], [ -72.36639404296875, -9.49444580078125 ], [ -72.305557250976562, -9.530000686645508 ], [ -72.297500610351562, -9.53639030456543 ], [ -72.253067016601562, -9.654167175292969 ], [ -72.251953125, -9.659723281860352 ], [ -72.2550048828125, -9.684446334838867 ], [ -72.258056640625, -9.695556640625 ], [ -72.26251220703125, -9.705833435058594 ], [ -72.26861572265625, -9.727779388427734 ], [ -72.270004272460938, -9.733333587646484 ], [ -72.2711181640625, -9.746667861938477 ], [ -72.263336181640625, -9.758335113525391 ], [ -72.260009765625, -9.761945724487305 ], [ -72.2469482421875, -9.769723892211914 ], [ -72.232223510742188, -9.776390075683594 ], [ -72.220840454101562, -9.779167175292969 ], [ -72.191680908203125, -9.79222297668457 ], [ -72.180007934570312, -9.801389694213867 ], [ -72.176956176757812, -9.805557250976562 ], [ -72.172500610351562, -9.814445495605469 ], [ -72.16473388671875, -9.834722518920898 ], [ -72.150848388671875, -9.882778167724609 ], [ -72.150283813476562, -9.888889312744141 ], [ -72.151123046875, -9.893611907958984 ], [ -72.1683349609375, -9.954444885253906 ], [ -72.185836791992188, -10.00111198425293 ], [ -72.185836791992188, -10.003889083862305 ], [ -72.143890380859375, -10.004722595214844 ], [ -72.052230834960938, -10.004722595214844 ], [ -72.010284423828125, -10.004446029663086 ], [ -71.979171752929688, -10.004446029663086 ], [ -71.930282592773438, -10.004446029663086 ], [ -71.641952514648438, -10.004446029663086 ], [ -71.45916748046875, -10.004167556762695 ], [ -71.368057250976562, -10.003889083862305 ], [ -71.29888916015625, -9.996389389038086 ], [ -71.293899536132812, -9.99444580078125 ], [ -71.1844482421875, -9.936944961547852 ], [ -71.180557250976562, -9.933612823486328 ], [ -71.172225952148438, -9.921390533447266 ], [ -71.161392211914062, -9.897222518920898 ], [ -71.156402587890625, -9.882778167724609 ], [ -71.151123046875, -9.873889923095703 ], [ -71.14056396484375, -9.863611221313477 ], [ -71.136123657226562, -9.860834121704102 ], [ -71.12640380859375, -9.856945037841797 ], [ -71.0211181640625, -9.824167251586914 ], [ -71.011398315429688, -9.820278167724609 ], [ -71.002227783203125, -9.814723968505859 ], [ -70.9908447265625, -9.805278778076172 ], [ -70.987228393554688, -9.801668167114258 ], [ -70.984451293945312, -9.797500610351562 ], [ -70.978897094726562, -9.782501220703125 ], [ -70.97222900390625, -9.768056869506836 ], [ -70.966400146484375, -9.759723663330078 ], [ -70.8819580078125, -9.670000076293945 ], [ -70.843612670898438, -9.645557403564453 ], [ -70.7791748046875, -9.605556488037109 ], [ -70.643341064453125, -9.490835189819336 ], [ -70.592788696289062, -9.443611145019531 ], [ -70.584457397460938, -9.438056945800781 ], [ -70.574447631835938, -9.434167861938477 ], [ -70.561676025390625, -9.432779312133789 ], [ -70.514663696289062, -9.428001403808594 ], [ -70.5452880859375, -9.500833511352539 ], [ -70.577224731445312, -9.538612365722656 ], [ -70.607589721679688, -9.575664520263672 ], [ -70.5977783203125, -9.61944580078125 ], [ -70.593612670898438, -9.629167556762695 ], [ -70.590835571289062, -9.633335113525391 ], [ -70.58111572265625, -9.644445419311523 ], [ -70.595291137695312, -9.781389236450195 ], [ -70.609451293945312, -9.795000076293945 ], [ -70.6158447265625, -9.802501678466797 ], [ -70.623062133789062, -9.816389083862305 ], [ -70.628067016601562, -9.835556030273438 ], [ -70.628890991210938, -9.895833969116211 ], [ -70.628616333007812, -9.948488235473633 ] ] ] ] } }
    ]
}

Overwriting drive/MyDrive/tests/vegMapper/smartin.geojson


>I made a new GeoJSON on [geojson.io](geojson.io) and copied it here to replace the one you sent to me on Slack, which didn't have any coincident points.

In [57]:
%%file drive/MyDrive/tests/vegMapper/smartin2.geojson
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -78.3984375,
              -5.298826889834371
            ],
            [
              -77.71728515624999,
              -5.943899579425587
            ],
            [
              -77.618408203125,
              -7.580327791330129
            ],
            [
              -76.739501953125,
              -8.602747284770006
            ],
            [
              -75.333251953125,
              -8.798225459016345
            ],
            [
              -74.970703125,
              -6.882800241767556
            ],
            [
              -75.21240234375,
              -5.8236866460048295
            ],
            [
              -75.52001953125,
              -4.795416925285452
            ],
            [
              -77.32177734375,
              -4.609278084409823
            ],
            [
              -77.684326171875,
              -4.488809196778639
            ],
            [
              -78.3984375,
              -5.298826889834371
            ]
          ]
        ]
      }
    }
  ]
}

Writing drive/MyDrive/tests/vegMapper/smartin2.geojson


In [58]:
# Set to 'None' to use minimum xy extent of input points:
relative_path_to_input_roi_in_drive = "drive/MyDrive/tests/vegMapper/smartin2.geojson"

os.path.isfile(relative_path_to_input_roi_in_drive)

True

In [59]:
# Get the minimum xt extent for the input points anyhow:
lon_min = pts['longitude'].min()
lon_max = pts['longitude'].max()
lat_min = pts['latitude'].min()
lat_max = pts['latitude'].max()

# Load an input GeoJSON by any means necessary:
if relative_path_to_input_roi_in_drive is None:
    roi_poly = [[lon_min, lat_max],
                [lon_min, lat_min],
                [lon_max, lat_min],
                [lon_max, lat_max]]
elif not os.path.isfile(relative_path_to_input_roi_in_drive):
    raise Exception("ERROR: No roi polygon provided.")
else:
    try:
        with open(relative_path_to_input_roi_in_drive, "r") as f:
            roi_poly = load(f)
    except Exception as e:
        raise e

print(list(roi_poly.keys()))

['type', 'features']


In [60]:
roi_feat = ee.Feature(roi_poly['features'][0])

roi = roi_feat.geometry()

type(roi)

ee.geometry.Geometry

Plot the region of interest polygon on a map to see the coverage.

In [61]:
roi_center = [pts['latitude'].mean(), pts['longitude'].mean()]

M = geemap.Map(center=roi_center, zoom=7, width="90%")
M.addLayer(roi, {'color': "red"}, name='ROI')
M.addLayer(pfc, name='Sites')
M

## Imagery

### Sentinel-1

We want to use preprocessed, analysis-ready data from the *S1_GRD* collection to calculate *radar volume index*.

>A previous version of this notebook used the *S1_GRD_FLOAT* collection (rather than the *S1_GRD* collection) because the data are in power units, and are thus immediately suitable to calculate *radar volume index* (as opposed to the *S1_GRD* collection, which gives the data in decibels (dB), i.e., on a logarithmic scale).
>
>See [this page](https://developers.google.com/earth-engine/guides/sentinel1) for more information about Sentinel-1 data accessible through GEE.

The next few cells take the following steps:

1. pulls target data from *both* S1 collections (*S1_GRD* and *S1_GRD_FLOAT*)
2. applies inverse transform to *S1_GRD* so they are represented in raw power
3. calculates summary statistics and verifies transform by comparing with data from *S1_GRD_FLOAT*

Get the data from the *S1_GRD_FLOAT* collection.

>This part that compares S1_GRD & S1_GRD_FLOAT is skipped if debug_status is False.

In [62]:
if debug_status():
    _s1 = (ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.eq("instrumentMode", "IW"))
            .filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING"))
            .filterDate(startDate, endDate)
            .filterBounds(roi)
            .select(['VV', 'VH'])
            .mean())
    print(type(_s1))  # this includes 113 images for our inputs on 2021-05-12
else:
    print(0)

0


The result should be a multi-band *Image*. Imagery were selected for our time/place of interest and composited to one mean raster per band in the output image.

>**Converting decibels to raw power**
>
>Imagery in the Earth Engine 'COPERNICUS/S1_GRD' Sentinel-1 ImageCollection is consists of Level-1 Ground Range Detected (GRD) scenes processed to backscatter coefficient (σ°) in decibels (dB). The backscatter coefficient represents target backscattering area (radar cross-section) per unit ground area. Because it can vary by several orders of magnitude, it is converted to dB as 10*log10σ°. It measures whether the radiated terrain scatters the incident microwave radiation preferentially away from the SAR sensor dB < 0) or towards the SAR sensor dB > 0). This scattering behavior depends on the physical characteristics of the terrain, primarily the geometry of the terrain elements and their electromagnetic characteristics.
>
>More info about this process may be found [here](https://developers.google.com/earth-engine/guides/sentinel1#sentinel-1-preprocessing) in the GEE docs.

Define and apply a function to do the inverse transform back into raw power units. Then do the same thing as in the cell above to select data from the *S1_GRD* collection, but apply the transform to all bands before getting the composite image.

In [63]:
def xform_s1(x):
    return ee.Image(10).pow(x.divide(10))

s1 = (ee.ImageCollection("COPERNICUS/S1_GRD")
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.eq("instrumentMode", "IW"))
        .filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING"))
        .filterDate(startDate, endDate)
        .filterBounds(roi)
        .select(['VV', 'VH'])
        .map(xform_s1)
        .mean())

type(s1)  # includes 113 images on 2021-05-12

ee.image.Image

>Stats skipped if debug_status is False.

Quick comparison between the two using the `gee.image_stats` convenience function. 

In [64]:
def get_stats(img, region=roi, scale=30):
    return geemap.image_stats(img=img, region=roi, scale=scale).getInfo()

if debug_status():
    display(HTML("<h3>S1_GRD_FLOAT</h3>"))
    display(pd.DataFrame(get_stats(_s1)))
    display(HTML("<h3>S1_GRD</h3>"))
    display(pd.DataFrame(get_stats(s1)))
else:
    print(0)

0


>*Assume the transform function is working properly if the values in both tables above are similar.*     
>If that's the case, we are good to proceed with the data from *S1_GRD*.

Dereference the data from the *S1_GRD_FLOAT* collection as we no longer need it.

In [65]:
_s1 = None

#### Calculate radar volume index

Add a new band to the output image containing the radar volume index calculated from *VV* and *VH*: `4 * VH / (VH + VV)`

In [66]:
s1out = s1.addBands(s1.expression(
    expression="4 * VH / (VH + VV)",
    opt_map={'VV': s1.select('VV'),
             'VH': s1.select('VH')}
).rename('radar_volume_index'))

type(s1out)

ee.image.Image

>Histograms + stats skipped if debug_status is False.

Plot the histograms for *VV* and *VH* (or just get simple summary stats like before).

In [67]:
# s1tmp = geemap.ee_to_numpy(s1comp, bands=["VV", "VH"], region=roi)

# # Plot histogram for VV:
# counts, bins = np.histogram(a=s1tmp[:,:,0].flatten())
# plt.hist(s1tmp[:,:,0], bins=bins)
# plt.ylim(0., 3.)
# plt.xlim(-18., -5.)
# plt.show()

# # Plot histogram for VH:
# counts, bins = np.histogram(a=s1tmp[:,:,1].flatten())
# plt.hist(s1tmp[:,:,1], bins=bins)
# plt.ylim(0., 3.)
# plt.xlim(-18., -5.)
# plt.show()

if debug_status():
    s1stats = get_stats(s1out)
    display(pd.DataFrame(s1stats))
else:
    print(0)

0


Now plot all the bands (including the computed *radar_volume_index* image/band) for visual inspection:

In [68]:
def drawMap(image, style_func, width="90%", **kwargs):
    M = geemap.Map(**kwargs)
    for band in image.bandNames().getInfo():
        M.addLayer(image.select(band), **style_func(band))
    M.addLayerControl()
    return M

def s1style(b: str, vis_params: dict={}):
    if b.endswith("radar_volume_index"):
        vis_params = {'min': s1stats['min'][b], 'max': s1stats['max'][b]}
    return {'vis_params': vis_params, 
            'shown': b.endswith("radar_volume_index"), 
            'name': b}

if debug_status():
    drawMap(image=s1out, style_func=s1style, center=roi_center, zoom=7, width="80%")
else:
    print(0)

0


### ALOS2

https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_SAR

This dataset from ALOS2 only has one timestep per year, so modify the start and end dates before applying *filterDate* to the *ImageCollection*.

In [69]:
years = [f'{startDate.split("-")[0]}-01-01', 
         f'{endDate.split("-")[0]}-12-31']

alos2 = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')
           .filterDate(*years)
           .filterBounds(roi)
           .select(['HV','HH'])
           .mean())

type(alos2)

ee.image.Image

#### Calculate radar volume index

Add a new band to the output image containing the ALOS2 radar volume index calculated as: `4 * HV / (HV + HH)`

In [70]:
alos2out = alos2.addBands(alos2.expression(
    expression="4 * HV / (HV + HH)", 
    opt_map={'HV': alos2.select('HV'),
             'HH': alos2.select('HH')}
).rename('radar_volume_index'))

type(alos2out)

ee.image.Image

Calculate and display a table of summary stats.

In [71]:
if debug_status():
    alos2stats = get_stats(alos2out)
    display(pd.DataFrame(alos2stats))
else:
    print(0)

0


### Landsat 8 SR

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR

>This example describes exactly what we need to do to produce the Landsat 8 median composite:    
https://developers.google.com/earth-engine/guides/ic_composite_mosaic

Select Landsat 8 surface reflectance images and apply a quality mask by mapping a function over all the images in the collection that match our filtering criteria. Then, calculate NDVI and add it to the image as a new band.

In [72]:
def mask_l8sr(image):
    cloudShadowBitMask = 1<<3
    cloudBitMask = 1<<5
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
           qa.bitwiseAnd(cloudBitMask).eq(0))
    return image.updateMask(mask)

# Select Landsat 8 surface reflectance scenes and make a median composite.
l8sr = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
          .filterDate(startDate, endDate)
          .filterBounds(roi)
          .map(mask_l8sr)
          .median())

# Use the composite bands 5 and 4 to produce an NDVI image.
l8out = l8sr.addBands(l8sr.normalizedDifference(['B5','B4']).rename('ndvi'))

type(l8out)

ee.image.Image

Draw NDVI and the other surface reflectance bands on a map.

In [73]:
def l8style(b: str, vis_params: dict={}):
    if b.endswith("ndvi"):
        vis_params = {'min': -1.0, 'max': 1.0, 'palette': 'red,yellow,green'}
    return {'vis_params': vis_params, 'shown': b.endswith("ndvi"), 'name': b}

if debug_status():
    drawMap(image=l8out, style_func=l8style, center=roi_center, zoom=7, width="75%")
else:
    print(0)

0


### VCF from MODIS

In [74]:
modis = (ee.ImageCollection('MODIS/006/MOD44B')
           .filterDate(*years)
           .filterBounds(roi)
           .select('Percent_Tree_Cover')
           .first())

type(modis)

ee.image.Image

## Run the prediction

Not clear on this so it's removed for now:

```python
prior_mean = [0.06491638, -26.63132179, 0.05590800, -29.64091620]
prior_mean_int = ee.Number(6.07)
prediction = (ee.Image(prior_mean_int)
              #.add((Fmod_tc_aoi.multiply(prior_mean[0]))
              .add(s1.select('radar_volume_index').multiply(prior_mean[1]))
              .add(l8sr.select('ndvi').multiply(prior_mean[2]))
              #.add(smooth.select('constant').multiply(prior_mean[3]))))
              ).clip(roi)
predx = prediction.exp()
pred_final = ee.Image(predx.divide(predx.add(1)))
type(pred_final)
```

### Build the stack

>Documentation for the image method *clipToBoundsAndScale* is helpful to understanding this step in GEE: https://developers.google.com/earth-engine/apidocs/ee-image-cliptoboundsandscale
>
>But I ended up using regular *clip* for now.
>
>Also see this information on compositing and image projections:      
>https://developers.google.com/earth-engine/guides/projections#the-default-projection

Configure preferences here to determine how the stack is created with a common grid. All imagery *added* to the stack using the *ee.Image.addBands* method will inherit the projection and scale of the parent image.

In [75]:
def prepare_image(image, pre: str, crs: str="EPSG:4326", scale: int=30):
    renamed = image.rename([f'{pre}-{b}' for b in image.bandNames().getInfo()])
    return renamed.setDefaultProjection(crs=crs, scale=scale).clip(roi)

s1out = prepare_image(image=s1out, pre="S1")
alos2out = prepare_image(image=alos2out, pre="ALOS2")
l8out = prepare_image(image=l8out, pre="L8")
modis = prepare_image(image=modis, pre="MODIS")
stack = None  # Assemble stack in a loop.
for i in [s1out, alos2out, l8out, modis]:
    if stack is not None:
        stack = stack.addBands(i)
    else:
        stack = i

type(stack)

ee.image.Image

Generate rough stats about all bands in the stack.

In [76]:
if debug_status():
    stack_stats = pd.DataFrame(get_stats(stack, scale=500))
    display(stack_stats)
else:
    print(0)

0


Get some information about the spatial properties of the stack.

In [77]:
if debug_status():
    stack.select("MODIS-Percent_Tree_Cover").projection().nominalScale().getInfo()
else:
    print(0)

0


Draw a map of all the bands with *geemap*.

In [78]:
def allstyle(b: str, vis_params: dict={}, shown: bool=False):
    if b.endswith("radar_volume_index"):
        vis_params = stack_stats.loc[b][['min','max']].to_dict()
        shown = True
    if b.endswith("ndvi"):
        vis_params = {'min': -1.0, 'max':  1.0, 'palette': 'red,yellow,green'}
        shown = True
    return {'name': b, 'viz_params': vis_params, 'shown': shown}

if debug_status():
    drawMap(image=stack, style_func=allstyle, center=roi_center, zoom=7)
else:
    print(0)

0


Verify spatial referencing information by displaying a dictionary of SRS information for each band.

In [79]:
if debug_status():
    srs = {}
    for b in stack.bandNames().getInfo():
        p = stack.select(b).projection()
        srs[b] = p.getInfo()
        srs[b]['nominalScale'] = p.nominalScale().getInfo()
        del srs[b]['type']
    # All image have identical projections if this test returns true:
    len(list(set([str(p) for p in list(srs.values())]))) == 1
else:
    print(0)

0


## Zonal statistics

Buffer the points Feature Collection. **This still needs more careful checks.** Appears OK though at first glance. (https://developers.google.com/earth-engine/apidocs/ee-feature-buffer)

>Swap the comments in this cell to *turn on and off the buffers around the xy locations* from the input csv. (Which indeed gives slightly different results, buffer vs no buffer.)

Map zonal stats over the feature collection after building the stack.

In [96]:
outputs = stack.reduceRegions(collection=pfc.map(lambda x: x.buffer(distance=buffer)),
                              #collection=pfc,
                              reducer=ee.Reducer.mean(),
                              #crs=stack.projection(),
                              scale=30)

type(outputs)

ee.featurecollection.FeatureCollection

Get the new `ee.FeatureCollection` as a dictionary then call the *pandas* convenience function `json_normalize` to translate to a `pandas.DataFrame`. 

In [97]:
outputs = pd.json_normalize(outputs.getInfo()['features'])

outputs.describe()

,properties.ALOS2-HH,properties.ALOS2-HV,properties.ALOS2-radar_volume_index,properties.L8-B1,properties.L8-B10,properties.L8-B11,properties.L8-B2,properties.L8-B3,properties.L8-B4,properties.L8-B5,properties.L8-B6,properties.L8-B7,properties.L8-ndvi,properties.L8-pixel_qa,properties.L8-radsat_qa,properties.L8-sr_aerosol,properties.MODIS-Percent_Tree_Cover,properties.S1-VH,properties.S1-VV,properties.S1-radar_volume_index
count,100.000000,100.000000,100.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.0,99.000000,100.000000,100.000000,100.000000,100.000000
mean,6252.613713,2591.864934,1.182499,330.831497,2937.316576,2905.785374,370.774299,632.008382,504.915847,3251.940714,1692.108171,844.848856,0.717178,322.134694,0.0,146.305072,42.731354,0.037061,0.192112,0.687422
std,2443.550671,1057.426647,0.249421,206.180231,19.586058,15.896990,232.090952,300.669581,366.523778,872.641797,523.898561,458.425787,0.223661,0.825747,0.0,49.690528,24.878543,0.020001,0.094561,0.207446
min,1149.765217,517.736934,0.431780,-283.863322,2847.115917,2854.354671,-162.655709,-24.399654,34.129758,432.022491,267.111979,166.513021,-0.171376,322.000000,0.0,73.277778,4.000000,0.002369,0.008091,0.195533
25%,4868.805587,2060.518229,1.056469,198.294726,2928.938203,2897.608580,212.392129,404.173813,234.786087,2752.032230,1365.234760,523.766899,0.588693,322.000000,0.0,96.000000,20.370017,0.028619,0.120346,0.562339
50%,6210.940513,2605.134913,1.221210,315.788428,2940.848958,2909.049037,313.309773,577.153846,380.852787,3427.500000,1612.584495,683.510363,0.820952,322.000000,0.0,150.914186,38.700881,0.037133,0.208353,0.660800
75%,7308.298883,3219.491663,1.311755,421.478674,2947.774033,2916.504766,451.474019,797.468165,715.269680,3784.645349,1959.595835,1022.024330,0.875886,322.000000,0.0,192.000000,66.161917,0.042258,0.235497,0.800926
max,18613.833910,6494.565744,1.731787,1066.532118,2986.478223,2939.570557,1160.858885,1668.050523,1712.485192,5362.180243,3316.280488,2586.967770,0.901852,329.123478,0.0,224.045296,83.000000,0.171241,0.670399,2.019049


Rename the columns according to the index in our *stack_stats* table from a few cells ago. (GEE appends the word "properties" fitting with common GIS convention.)

>We could put some automated + hands-on validation routines at the bottom of the ipynb, e.g. a row-picker to render this table and a map widget next to it.

Here's the first row of data after renaming the columns:

In [98]:
outputs_names = {f"properties.{i}": i for i in outputs.index.tolist()}

outputs = outputs.rename(mapper=outputs_names, axis=1)

outputs.iloc[0].to_frame(name="ROW_0")

,ROW_0
type,Feature
id,0
geometry.type,Polygon
geometry.coordinates,"[[[-76.54613982, -8.219771455021448], [-76.546..."
properties.ALOS2-HH,10784.6
properties.ALOS2-HV,1281.82
properties.ALOS2-radar_volume_index,0.498507
properties.L8-B1,642.64
properties.L8-B10,2960.11
properties.L8-B11,2925.35


## Outputs

### Save to Google Drive

>*Important: Make sure to give a path that's inside the Drive directory.*

Write to Google Drive with the `to_csv` method.

In [99]:
outfile = os.path.join("drive/MyDrive/", relative_path_to_output_csv_in_drive)
outpath = os.path.dirname(outfile)

!mkdir -p $outpath

outputs.to_csv(outfile, index=None)

drive.flush_and_unmount()  # remember to nmount Drive when youre done.

Drive not mounted, so nothing to flush and unmount.


### Download to local disk

Run this next cell to save to your local machine as a CSV.

In [100]:
outputs.to_csv("outputs.csv", index=None)  # write to drive/colab workspace

files.download(filename="outputs.csv")  # write to local disk

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## References

* https://developers.google.com/earth-engine/guides/resample#resampling
* https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points#understanding_which_pixels_are_included_in_polygon_statistics
  * https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points#notes_on_crs_and_scale
* https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points#zonalstatsfc_params_%E2%87%92_eefeaturecollection
* https://developers.google.com/earth-engine/tutorials/community/beginners-cookbook#example_exporting_data

## Notes

Important concepts in GEE:

* Scale: https://developers.google.com/earth-engine/guides/scale
* Projections: https://developers.google.com/earth-engine/guides/projections
  * *The default projection*: https://developers.google.com/earth-engine/guides/projections#the-default-projection
  * *Composites have no projection*: https://developers.google.com/earth-engine/guides/ic_reducing#Composites-have-no-projection

I was wary of using `reproject` at first because of how it's described in [the GEE documentation](https://developers.google.com/earth-engine/guides/projections#reprojecting), but now I see that it's a must to achieve the common grid. (GEE does everything else for me in a sensible way _EXCEPT_ for this, IMO.)

* https://developers.google.com/earth-engine/guides/image_math#colab-python_1